# Алгоритм Нидлмана-Вунша

In [1]:
from IPython.display import HTML, display
import numpy as np

def display_table(data):
    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td><h4>%s</h4><td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

Посмотрим, как будут меняться выравнивания в зависимости от матрицы весов на примере следующих последовательностей:

In [11]:
a = "GGCTCGTTATGCATGGCCGAGCA"
b = "AGAGATTTAGCAAACCCTCTAACAGCC"
match = 1
gap = -1
mismatch = -1

Для получения первых выравниваний использовалась следующая матрица весов:

In [12]:
data = [["-","A","C","G","T"],["A",1,-1,-1,-1],["C",-1,1,-1,-1],["G",-1,-1,1,-1],["T",-1,-1,-1,1]]
display_table(data)

И соотвествующая функция score()

In [13]:
def score(x,y):
    if x == y:
        return match
    if x == "_" or y == "_":
        return gap
    return mismatch

Будем использовать новую матрицу весов и посмотрим, как изменятся выравнивания:

In [14]:
data = [["-","A","C","G","T"],["A",8,-1,-1,-1],["C",-1,9,-1,-1],["G",-1,-1,7,-1],["T",-1,-1,-1,10]]
display_table(data)

Этой матрице будет соответствовать новая функция score_new()

In [15]:
def score_new(x,y):
    if x == "T" and y == "T":
        return 10
    if x == "A" and y == "A":
        return 8
    if x == "C" and y == "C":
        return 9
    if x == "G" and y == "G":
        return 7
    if x == "_" or y == "_":
        return gap
    if x != y:
        return mismatch

Изменим штраф за несовпадение A и С и снова посмотрим, как изменятся выравнивания.

In [16]:
data = [["-","A","C","G","T"],["A",8,-3,-1,-1],["C",-3,9,-1,-1],["G",-1,-1,7,-1],["T",-1,-1,-1,10]]
display_table(data)

In [17]:
def score_upd(x,y):
    if x == "T" and y == "T":
        return 10
    if x == "A" and y == "A":
        return 8
    if x == "C" and y == "C":
        return 9
    if x == "G" and y == "G":
        return 7
    if (x == "A" and y == "C") or (x == "C" and y == "A"):
        return -3
    if x == "_" or y == "_":
        return gap
    if x != y:
        return mismatch

Ниже представлен сам алгоритм выравнивания и результаты его работы на разных матрицах весов:

In [18]:
def needleman_wunsch(a, b, score_fun):
    n, m = len(a), len(b)
    match = 1
    gap = -1
    mismatch = -1
    mat = np.zeros((m + 1, n + 1))
    
    for i in range(0, m + 1):
            mat[i][0] = gap * i
    for j in range(0, n + 1):
            mat[0][j] = gap * j
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            top = mat[i-1][j] + gap
            left = mat[i][j-1] + gap
            diag = mat[i-1][j-1] + score_fun(a[j-1],b[i-1])
            mat[i][j] = max(diag, left , top)      
    seq1 = ""
    seq2 = ""

    i = m
    j = n
    while i > 0 and j > 0:
        score_curr = mat[i][j]
        score_top = mat[i][j-1]
        score_left = mat[i-1][j]
        score_diag = mat[i-1][j-1]
        if score_curr == score_diag + score_fun(a[j-1], b[i-1]):
            seq1 = a[j-1] + seq1
            seq2 = b[i-1] + seq2
            i -= 1
            j -= 1
        elif score_curr == score_top + gap:
            seq1 = a[j-1] + seq1
            seq2 = '_' + seq2
            j -= 1
        elif score_curr == score_left + gap:
            seq1 = '_' + seq1
            seq2 = b[i-1] + seq2
            i -= 1
    while j > 0:
        seq1 = a[j-1] + seq1
        seq2 = '_' + seq2
        j -= 1
    while i > 0:
        seq1 = '_' + seq1
        seq2 = b[i-1] + seq2
        i -= 1


    print(seq1, seq2, sep = "\n")

In [19]:
needleman_wunsch(a, b, score)

_G_GCTCGTTATGC_ATGGC_C___GAGCA
AGAGAT__TTA_GCAAACCCTCTAACAGCC


In [20]:
needleman_wunsch(a, b, score_new)

_G_GCTCGTTATGC__A___TGGC___CGAGCA
AGAGAT__TTA_GCAAACCCT__CTAAC_AGCC


In [21]:
needleman_wunsch(a, b, score_upd)

_G_G_CTCGTTATGC__A___TGGC___CGAG_CA
AGAGA_T__TTA_GCAAACCCT__CTAAC_AGCC_
